In [10]:
"SGLB 1 model"

'SGLB 1 model'

In [1]:
import math
import numpy as np
import argparse
import sys
from scipy.stats import norm
def write(X, y, file):
    with open(file, "w") as f:
        for i in range(len(y)):
            f.write("{y}\t{y}\t{x}\n".format(y=y[i], x='\t'.join(map(str, X[i]))))
def writeOOD(X, Y, file):
    y = []
    with open(file, "w") as f:
        for i in range(len(X)):
            y.append(0)
            f.write("{y}\t{y}\t{x}\n".format(y=0, x='\t'.join(map(str, X[i]))))
        for i in range(len(Y)):
            y.append(1)
            f.write("{y}\t{y}\t{x}\n".format(y=1, x='\t'.join(map(str, Y[i]))))
    return np.array(y)
from sklearn.metrics import auc

from sklearn.metrics import roc_auc_score
def Auc(x, y):
    order = np.lexsort((y, x))
    x, y = np.array(x)[order], np.array(y)[order]
    return auc(x, y)


def reject_MSE(targets, preds, measure, measure_name, save_path, pos_label=1, show=True):
    if pos_label != 1:
        measure_loc = -1.0 * measure
    else:
        measure_loc = measure
    preds = np.squeeze(preds)
    # Compute total MSE
    error = (preds - targets)**2
    MSE_0 = np.mean(error)
    # print 'BASE MSE', MSE_0

    # Create array
    array = np.concatenate(
        (preds[:, np.newaxis], targets[:, np.newaxis], error[:, np.newaxis], measure_loc[:, np.newaxis]), axis=1)

    # Results arrays
    results_max = [[0.0, 0.0]]
    results_var = [[0.0, 0.0]]
    results_min = [[0.0, 0.0]]

    optimal_ranking = array[:, 2].argsort()
    sorted_array = array[optimal_ranking]  # Sort by error

    for i in range(1, array.shape[0]):
        x = np.concatenate((sorted_array[:-i, 0], sorted_array[-i:, 1]), axis=0)
        mse = np.mean((x - sorted_array[:, 1])**2)
        # Best rejection
        results_max.append([float(i) / float(array.shape[0]), (MSE_0 - mse) / MSE_0])
        # Random Rejection
        results_min.append([float(i) / float(array.shape[0]), float(i) / float(array.shape[0])])

    uncertainty_ranking = array[:, 3].argsort()
    sorted_array = array[uncertainty_ranking]  # Sort by uncertainty

    for i in range(1, array.shape[0]):
        x = np.concatenate((sorted_array[:-i, 0], sorted_array[-i:, 1]), axis=0)
        mse = np.mean((x - sorted_array[:, 1]) ** 2)
        results_var.append([float(i) / float(array.shape[0]), (MSE_0 - mse) / MSE_0])

    max_auc = Auc([x[0] for x in results_max], [x[1] for x in results_max])
    var_auc = Auc([x[0] for x in results_var], [x[1] for x in results_var])
    min_auc = Auc([x[0] for x in results_min], [x[1] for x in results_min])

    AUC_RR = (var_auc - min_auc) / (max_auc - min_auc)
    return np.sqrt(MSE_0), AUC_RR, min_auc, var_auc, max_auc
def reject_MSE1(targets, preds, measure, measure_name, save_path, pos_label=1, show=True):
    return 0.0, roc_auc_score(targets, measure), 0.0, 0.0, 0.0
#     return np.sqrt(MSE_0), AUC_RR, min_auc, var_auc, max_auc
approach = "sglb"
params_dict = {}
results_dict = dict()
for data_directory in ["bostonHousing",  "kin8nm", "protein-tertiary-structure", "concrete", "naval-propulsion-plant",  "wine-quality-red", "energy", "power-plant", "yacht"]:
    from subprocess import call
  
    _DATA_DIRECTORY_PATH = "./UCI_Datasets/" + data_directory + "/data/"
    _DROPOUT_RATES_FILE = _DATA_DIRECTORY_PATH + "dropout_rates.txt"
    _TAU_VALUES_FILE = _DATA_DIRECTORY_PATH + "tau_values.txt"
    _DATA_FILE = _DATA_DIRECTORY_PATH + "data.txt"
    _DATA_FILE_OOD = _DATA_DIRECTORY_PATH + "data-mix-ood.txt"
    _HIDDEN_UNITS_FILE = _DATA_DIRECTORY_PATH + "n_hidden.txt"
    _EPOCHS_FILE = _DATA_DIRECTORY_PATH + "n_epochs.txt"
    _INDEX_FEATURES_FILE = _DATA_DIRECTORY_PATH + "index_features.txt"
    _INDEX_TARGET_FILE = _DATA_DIRECTORY_PATH + "index_target.txt"
    _N_SPLITS_FILE = _DATA_DIRECTORY_PATH + "n_splits.txt"
    def _get_index_train_test_path(split_num, train = True):
        """
           Method to generate the path containing the training/test split for the given
           split number (generally from 1 to 20).
           @param split_num      Split number for which the data has to be generated
           @param train          Is true if the data is training data. Else false.
           @return path          Path of the file containing the requried data
        """
        if train:
            return _DATA_DIRECTORY_PATH + "index_train_" + str(split_num) + ".txt"
        else:
            return _DATA_DIRECTORY_PATH + "index_test_" + str(split_num) + ".txt" 


    # We fix the random seed

    np.random.seed(1)

    print ("Loading data and other hyperparameters...")
    # We load the data

    data = np.loadtxt(_DATA_FILE)
    data_ood = np.loadtxt(_DATA_FILE_OOD)


    # We load the indexes for the features and for the target

    index_features = np.loadtxt(_INDEX_FEATURES_FILE)
    index_target = np.loadtxt(_INDEX_TARGET_FILE)

    X = data[ : , [int(i) for i in index_features.tolist()] ]
    Xood = data_ood [1: : , [int(i) for i in index_features.tolist()]]
    y = data[ : , int(index_target.tolist()) ]

    # We iterate over the training test splits

    n_splits = np.loadtxt(_N_SPLITS_FILE)
    print ("Done.")

    errors, MC_errors, lls = [], [], []
    for split in range(int(n_splits)):
        print("split {} of {}".format(split, n_splits))
        # We load the indexes of the training and test sets
        print ('Loading file: ' + _get_index_train_test_path(split, train=True))
        print ('Loading file: ' + _get_index_train_test_path(split, train=False))
        index_train = np.loadtxt(_get_index_train_test_path(split, train=True))
        index_test = np.loadtxt(_get_index_train_test_path(split, train=False))

        X_train = X[ [int(i) for i in index_train.tolist()] ]
        y_train = y[ [int(i) for i in index_train.tolist()] ]

        X_test = X[ [int(i) for i in index_test.tolist()] ]
        y_test = y[ [int(i) for i in index_test.tolist()] ]

        X_train_original = X_train
        y_train_original = y_train
        num_training_examples = int(0.8 * X_train.shape[0])
        X_validation = X_train[num_training_examples:, :]
        y_validation = y_train[num_training_examples:]
        X_train = X_train[0:num_training_examples, :]
        y_train = y_train[0:num_training_examples]
        y_train_mean = y_train.mean()
        y_train_std = y_train.std(ddof=0)
        y_train_normalized = y_train - y_train_mean
        y_validate_normalized = y_validation - y_train_mean
        write(X_train, y_train_normalized, "_train.tsv")
        write(X_validation, y_validate_normalized, "_validation.tsv")
        N = len(y_train)
        beta = N + 0.
        shrinkage = 1.0 / N
        num_ensembles = 1
        num = 1
        # Printing the size of the training, validation and test sets
        print ('Number of training examples: ' + str(X_train.shape[0]))
        print ('Number of validation examples: ' + str(X_validation.shape[0]))
        print ('Number of test examples: ' + str(X_test.shape[0]))
        print ('Number of train_original examples: ' + str(X_train_original.shape[0]))

        # List of hyperparameters which we will try out using grid-search
        
        best_depth = params_dict[data_directory][split]["depth"]
        best_lr = params_dict[data_directory][split]["lr"]
        best_rs = params_dict[data_directory][split]["rs"]
        best_ll = float("-inf")
        best_iters = params_dict[data_directory][split]["iters"]
        
        X_train = X_train_original
        y_train = y_train_original
        y_train_mean = y_train.mean()
        y_train_std = y_train.std(ddof=0)
        y_train_normalized = y_train - y_train_mean
        y_test_normalized = y_test - y_train_mean
        write(X_train, y_train_normalized, "_train.tsv")
        write(X_test, y_test_normalized, "_test.tsv")
        yood = writeOOD(X_test, Xood, "_ood.tsv")
        N = len(y_train)
        beta = N + 0.
        shrinkage = 1.0 / N
        predictions = []
        for index in range(num_ensembles):
            !./catboost fit -f _train.tsv -t _test.tsv --cd pool.cd  --bootstrap-type Bernoulli --subsample 0.5 --use-best-model False --depth $best_depth --learning-rate $best_lr --random-strength $best_rs --leaf-estimation-method Newton --loss-function MultiRMSE:mean=0\;std=$y_train_std --iterations $best_iters --seed $index -m out_.m > output__.tsv
            for i in range(num):
                trees = best_iters - i
                !./catboost calc -m out_.m --input-path _ood.tsv -o out_.tsv --cd pool.cd --prediction-type RawFormulaVal --tree-count-limit $trees > stdoutapply.txt
                with open("out_.tsv") as f:
                    tmp = [x.strip().split('\t') for x in f.readlines()[1:]]
                    CC = (1 - best_lr * shrinkage) ** (-i)
                    predictions.append([[CC * float(x[1]), np.exp(np.log(y_train_std) + CC * float(x[2]))] for x in tmp])
        ll = 0.0
        for index in range(len(y_test)):
            p = 0.0
            yy = y_test_normalized[index]
            for i in range(max(num_ensembles, num)):
                p += norm.pdf((yy - predictions[i][index][0]) / predictions[i][index][1]) / predictions[i][index][1] / max(num_ensembles, num)
            ll += (np.log(num) - np.log(p+1e-6)) / len(y_test)
        preds = np.array([np.mean([x[y][0] for x in predictions]) for y in range(len(y_test))])
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])]) + np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])])
        mse, rmse1, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse2, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        means = np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])
        stds = np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])
        epkl = np.array([np.sum([0.5 * ((s2 ** 2 / s1 ** 2) + (m1-m2)**2 / s1 ** 2 - 1 + 2*np.log(s2/s1)) for m1, s1 in zip(x1, y1) for m2, s2 in zip(x1, y1)]) / len(x1) / max(1, len(x1) - 1) for x1, y1 in zip(means, stds)])
        mse, rmse4, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, epkl, "variance", "anywhere")
        ent = np.array([np.mean(0.5 * np.log(x)) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse3, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, ent, "variance", "anywhere")
        ub = epkl + ent
        mse, rmse5, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, ub, "variance", "anywhere")
        
        y_test = yood
        y_test_normalized = yood
        preds = np.array([np.mean([x[y][0] for x in predictions]) for y in range(len(y_test))])
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])]) + np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse6, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])])
        _, rmse7, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse8, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        means = np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])
        stds = np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])
        epkl = np.array([np.sum([0.5 * ((s2 ** 2 / s1 ** 2) + (m1-m2)**2 / s1 ** 2 - 1 + 2*np.log(s2/s1)) for m1, s1 in zip(x1, y1) for m2, s2 in zip(x1, y1)]) / len(x1) / max(1, len(x1) - 1) for x1, y1 in zip(means, stds)])
        _, rmse10, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, epkl, "variance", "anywhere")
        ent = np.array([np.mean(0.5 * np.log(x)) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse9, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, ent, "variance", "anywhere")
        ub = epkl + ent
        _, rmse11, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, ub, "variance", "anywhere")
        
        print(mse, ll)
        errors.append(mse)
        MC_errors.append([rmse, rmse1, rmse2, rmse3, rmse4, rmse5, rmse6, rmse7, rmse8, rmse9, rmse10, rmse11, rmse11])
        print([rmse, rmse1, rmse2, rmse3, rmse4, rmse5, rmse6, rmse7, rmse8, rmse9, rmse10, rmse11, rmse11])
        lls.append(ll)
    results_dict[data_directory] = (errors, MC_errors, lls)
        

Loading data and other hyperparameters...
Done.
split 0 of 20.0
Loading file: ./UCI_Datasets/bostonHousing/data/index_train_0.txt
Loading file: ./UCI_Datasets/bostonHousing/data/index_test_0.txt
Number of training examples: 364
Number of validation examples: 91
Number of test examples: 51
Number of train_original examples: 455
2.483174141618647 2.380846234344451
[0.017385769623271256, 0.17584521157671676, 0.017385769623271256, 0.017385769623271256, 0.17584521157671676, 0.017385769623271256, 0.7612456747404844, 0.5, 0.7612456747404844, 0.7612456747404844, 0.5, 0.7612456747404844, 0.7612456747404844]
split 1 of 20.0
Loading file: ./UCI_Datasets/bostonHousing/data/index_train_1.txt
Loading file: ./UCI_Datasets/bostonHousing/data/index_test_1.txt
Number of training examples: 364
Number of validation examples: 91
Number of test examples: 51
Number of train_original examples: 455
4.634269882458993 2.9128772145661026
[0.7046880428029056, -0.1260385858149896, 0.7046880428029056, 0.704688042802

In [2]:
results_dict

{'bostonHousing': ([2.483174141618647,
   4.634269882458993,
   2.488575920974487,
   2.7296544795979996,
   4.250169868499084,
   2.604248365533571,
   1.8465820648569766,
   3.4971712063113687,
   3.886724154350776,
   4.51265364171326,
   3.4354964036734916,
   2.6443451511287788,
   2.8294862122615485,
   3.330083928838517,
   2.90391276108449,
   2.8967184996325996,
   2.896049455424853,
   3.0656629607722294,
   2.7022972167473727,
   4.002504216577501],
  [[0.017385769623271256,
    0.17584521157671676,
    0.017385769623271256,
    0.017385769623271256,
    0.17584521157671676,
    0.017385769623271256,
    0.7612456747404844,
    0.5,
    0.7612456747404844,
    0.7612456747404844,
    0.5,
    0.7612456747404844,
    0.7612456747404844],
   [0.7046880428029056,
    -0.1260385858149896,
    0.7046880428029056,
    0.7046880428029056,
    -0.1260385858149896,
    0.7046880428029056,
    0.5040369088811996,
    0.5,
    0.5040369088811996,
    0.5040369088811996,
    0.5,
    0.

In [6]:
"SGB 10 models"

'SGB 10 models'

In [3]:
import math
import numpy as np
import argparse
import sys
from scipy.stats import norm
def write(X, y, file):
    with open(file, "w") as f:
        for i in range(len(y)):
            f.write("{y}\t{y}\t{x}\n".format(y=y[i], x='\t'.join(map(str, X[i]))))
def writeOOD(X, Y, file):
    y = []
    with open(file, "w") as f:
        for i in range(len(X)):
            y.append(0)
            f.write("{y}\t{y}\t{x}\n".format(y=0, x='\t'.join(map(str, X[i]))))
        for i in range(len(Y)):
            y.append(1)
            f.write("{y}\t{y}\t{x}\n".format(y=1, x='\t'.join(map(str, Y[i]))))
    return np.array(y)
from sklearn.metrics import auc

from sklearn.metrics import roc_auc_score
def Auc(x, y):
    order = np.lexsort((y, x))
    x, y = np.array(x)[order], np.array(y)[order]
    return auc(x, y)


def reject_MSE(targets, preds, measure, measure_name, save_path, pos_label=1, show=True):
    if pos_label != 1:
        measure_loc = -1.0 * measure
    else:
        measure_loc = measure
    preds = np.squeeze(preds)
    # Compute total MSE
    error = (preds - targets)**2
    MSE_0 = np.mean(error)
    # print 'BASE MSE', MSE_0

    # Create array
    array = np.concatenate(
        (preds[:, np.newaxis], targets[:, np.newaxis], error[:, np.newaxis], measure_loc[:, np.newaxis]), axis=1)

    # Results arrays
    results_max = [[0.0, 0.0]]
    results_var = [[0.0, 0.0]]
    results_min = [[0.0, 0.0]]

    optimal_ranking = array[:, 2].argsort()
    sorted_array = array[optimal_ranking]  # Sort by error

    for i in range(1, array.shape[0]):
        x = np.concatenate((sorted_array[:-i, 0], sorted_array[-i:, 1]), axis=0)
        mse = np.mean((x - sorted_array[:, 1])**2)
        # Best rejection
        results_max.append([float(i) / float(array.shape[0]), (MSE_0 - mse) / MSE_0])
        # Random Rejection
        results_min.append([float(i) / float(array.shape[0]), float(i) / float(array.shape[0])])

    uncertainty_ranking = array[:, 3].argsort()
    sorted_array = array[uncertainty_ranking]  # Sort by uncertainty

    for i in range(1, array.shape[0]):
        x = np.concatenate((sorted_array[:-i, 0], sorted_array[-i:, 1]), axis=0)
        mse = np.mean((x - sorted_array[:, 1]) ** 2)
        results_var.append([float(i) / float(array.shape[0]), (MSE_0 - mse) / MSE_0])

    max_auc = Auc([x[0] for x in results_max], [x[1] for x in results_max])
    var_auc = Auc([x[0] for x in results_var], [x[1] for x in results_var])
    min_auc = Auc([x[0] for x in results_min], [x[1] for x in results_min])

#     plt.scatter([x[0] for x in results_max], [x for x in np.asarray(sorted(measure_loc, reverse=True))])
#     plt.xlim(0.0, 1.0)
#     if show == True:
#         plt.savefig(os.path.join(save_path, measure_name), bbox_inches='tight')
#     plt.close()
#     plt.plot([x[0] for x in results_max], [x[1] for x in results_max], '^',
#              [x[0] for x in results_var], [x[1] for x in results_var], 'o',
#              [x[0] for x in results_min], [x[1] for x in results_min], '--')
#     plt.legend(['Optimal-Rejection', 'Model-Rejection', 'Expected Random-Rejection'], loc=4)
#     plt.xlim(0.0, 1.0)
#     plt.ylim(0.0, 1.0)
#     plt.xlabel('Rejection Fraction')
#     plt.ylabel('Pearson Correlation')
#     if show==True:
#         plt.savefig(os.path.join(save_path, "MSE Rejection Curve using " + measure_name), bbox_inches='tight')
#     plt.close()

    AUC_RR = (var_auc - min_auc) / (max_auc - min_auc)
#     if save_path is not None:
#         with open(os.path.join(save_path, 'results.txt'), 'a') as f:
#             f.write('MSE ROC using ' + measure_name + ": " + str(np.round(AUC_RR * 100.0, 1)) + '\n')
    return np.sqrt(MSE_0), AUC_RR, min_auc, var_auc, max_auc
def reject_MSE1(targets, preds, measure, measure_name, save_path, pos_label=1, show=True):
    return 0.0, roc_auc_score(targets, measure), 0.0, 0.0, 0.0
#     return np.sqrt(MSE_0), AUC_RR, min_auc, var_auc, max_auc
approach = "sglb"
params_dict = {}
results_dict = dict()
for data_directory in ["bostonHousing",  "kin8nm", "protein-tertiary-structure", "concrete", "naval-propulsion-plant",  "wine-quality-red", "energy", "power-plant", "yacht"]:
    from subprocess import call
  
    _DATA_DIRECTORY_PATH = "./UCI_Datasets/" + data_directory + "/data/"
    _DROPOUT_RATES_FILE = _DATA_DIRECTORY_PATH + "dropout_rates.txt"
    _TAU_VALUES_FILE = _DATA_DIRECTORY_PATH + "tau_values.txt"
    _DATA_FILE = _DATA_DIRECTORY_PATH + "data.txt"
    _DATA_FILE_OOD = _DATA_DIRECTORY_PATH + "data-mix-ood.txt"
    _HIDDEN_UNITS_FILE = _DATA_DIRECTORY_PATH + "n_hidden.txt"
    _EPOCHS_FILE = _DATA_DIRECTORY_PATH + "n_epochs.txt"
    _INDEX_FEATURES_FILE = _DATA_DIRECTORY_PATH + "index_features.txt"
    _INDEX_TARGET_FILE = _DATA_DIRECTORY_PATH + "index_target.txt"
    _N_SPLITS_FILE = _DATA_DIRECTORY_PATH + "n_splits.txt"
    def _get_index_train_test_path(split_num, train = True):
        """
           Method to generate the path containing the training/test split for the given
           split number (generally from 1 to 20).
           @param split_num      Split number for which the data has to be generated
           @param train          Is true if the data is training data. Else false.
           @return path          Path of the file containing the requried data
        """
        if train:
            return _DATA_DIRECTORY_PATH + "index_train_" + str(split_num) + ".txt"
        else:
            return _DATA_DIRECTORY_PATH + "index_test_" + str(split_num) + ".txt" 


    # We fix the random seed

    np.random.seed(1)

    print ("Loading data and other hyperparameters...")
    # We load the data

    data = np.loadtxt(_DATA_FILE)
    data_ood = np.loadtxt(_DATA_FILE_OOD)


    # We load the indexes for the features and for the target

    index_features = np.loadtxt(_INDEX_FEATURES_FILE)
    index_target = np.loadtxt(_INDEX_TARGET_FILE)

    X = data[ : , [int(i) for i in index_features.tolist()] ]
    Xood = data_ood [1: : , [int(i) for i in index_features.tolist()]]
    y = data[ : , int(index_target.tolist()) ]

    # We iterate over the training test splits

    n_splits = np.loadtxt(_N_SPLITS_FILE)
    print ("Done.")

    errors, MC_errors, lls = [], [], []
    for split in range(int(n_splits)):
        print("split {} of {}".format(split, n_splits))
        # We load the indexes of the training and test sets
        print ('Loading file: ' + _get_index_train_test_path(split, train=True))
        print ('Loading file: ' + _get_index_train_test_path(split, train=False))

        X_train_original = X_train
        y_train_original = y_train
        num_training_examples = int(0.8 * X_train.shape[0])
        X_validation = X_train[num_training_examples:, :]
        y_validation = y_train[num_training_examples:]
        X_train = X_train[0:num_training_examples, :]
        y_train = y_train[0:num_training_examples]
        y_train_mean = y_train.mean()
        y_train_std = y_train.std(ddof=0)
        y_train_normalized = y_train - y_train_mean
        y_validate_normalized = y_validation - y_train_mean
        write(X_train, y_train_normalized, "_train.tsv")
        write(X_validation, y_validate_normalized, "_validation.tsv")
        N = len(y_train)
        beta = N + 0.
        shrinkage = 1.0 / N
        num_ensembles = 10
        num = 1
        # Printing the size of the training, validation and test sets
        print ('Number of training examples: ' + str(X_train.shape[0]))
        print ('Number of validation examples: ' + str(X_validation.shape[0]))
        print ('Number of test examples: ' + str(X_test.shape[0]))
        print ('Number of train_original examples: ' + str(X_train_original.shape[0]))

        # List of hyperparameters which we will try out using grid-search
        
        best_depth = params_dict[data_directory][split]["depth"]
        best_lr = params_dict[data_directory][split]["lr"]
        best_rs = params_dict[data_directory][split]["rs"]
        best_ll = float("-inf")
        best_iters = params_dict[data_directory][split]["iters"]
        
        X_train = X_train_original
        y_train = y_train_original
        y_train_mean = y_train.mean()
        y_train_std = y_train.std(ddof=0)
        y_train_normalized = y_train - y_train_mean
        y_test_normalized = y_test - y_train_mean
        write(X_train, y_train_normalized, "_train.tsv")
        write(X_test, y_test_normalized, "_test.tsv")
        yood = writeOOD(X_test, Xood, "_ood.tsv")
        N = len(y_train)
        beta = N + 0.
        shrinkage = 1.0 / N
        predictions = []
        for index in range(num_ensembles):
            !./catboost fit -f _train.tsv -t _test.tsv --cd pool.cd  --bootstrap-type Bernoulli --subsample 0.5 --use-best-model False --depth $best_depth --learning-rate $best_lr --random-strength $best_rs --leaf-estimation-method Newton --loss-function MultiRMSE:mean=0\;std=$y_train_std --iterations $best_iters --seed $index -m out_.m > output__.tsv
            for i in range(num):
                trees = best_iters - i
                !./catboost calc -m out_.m --input-path _ood.tsv -o out_.tsv --cd pool.cd --prediction-type RawFormulaVal --tree-count-limit $trees > stdoutapply.txt
                with open("out_.tsv") as f:
                    tmp = [x.strip().split('\t') for x in f.readlines()[1:]]
                    CC = (1 - best_lr * shrinkage) ** (-i)
                    predictions.append([[CC * float(x[1]), np.exp(np.log(y_train_std) + CC * float(x[2]))] for x in tmp])
        ll = 0.0
        for index in range(len(y_test)):
            p = 0.0
            yy = y_test_normalized[index]
            for i in range(max(num_ensembles, num)):
                p += norm.pdf((yy - predictions[i][index][0]) / predictions[i][index][1]) / predictions[i][index][1] / max(num_ensembles, num)
            ll += (np.log(num) - np.log(p+1e-6)) / len(y_test)
        preds = np.array([np.mean([x[y][0] for x in predictions]) for y in range(len(y_test))])
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])]) + np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])])
        mse, rmse1, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse2, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        means = np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])
        stds = np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])
        epkl = np.array([np.sum([0.5 * ((s2 ** 2 / s1 ** 2) + (m1-m2)**2 / s1 ** 2 - 1 + 2*np.log(s2/s1)) for m1, s1 in zip(x1, y1) for m2, s2 in zip(x1, y1)]) / len(x1) / max(1, len(x1) - 1) for x1, y1 in zip(means, stds)])
        mse, rmse4, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, epkl, "variance", "anywhere")
        ent = np.array([np.mean(0.5 * np.log(x)) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse3, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, ent, "variance", "anywhere")
        ub = epkl + ent
        mse, rmse5, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, ub, "variance", "anywhere")
        
        y_test = yood
        y_test_normalized = yood
        preds = np.array([np.mean([x[y][0] for x in predictions]) for y in range(len(y_test))])
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])]) + np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse6, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])])
        _, rmse7, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse8, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        means = np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])
        stds = np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])
        epkl = np.array([np.sum([0.5 * ((s2 ** 2 / s1 ** 2) + (m1-m2)**2 / s1 ** 2 - 1 + 2*np.log(s2/s1)) for m1, s1 in zip(x1, y1) for m2, s2 in zip(x1, y1)]) / len(x1) / max(1, len(x1) - 1) for x1, y1 in zip(means, stds)])
        _, rmse10, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, epkl, "variance", "anywhere")
        ent = np.array([np.mean(0.5 * np.log(x)) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse9, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, ent, "variance", "anywhere")
        ub = epkl + ent
        _, rmse11, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, ub, "variance", "anywhere")
        
        print(mse, ll)
        errors.append(mse)
        MC_errors.append([rmse, rmse1, rmse2, rmse3, rmse4, rmse5, rmse6, rmse7, rmse8, rmse9, rmse10, rmse11, rmse11])
        print([rmse, rmse1, rmse2, rmse3, rmse4, rmse5, rmse6, rmse7, rmse8, rmse9, rmse10, rmse11, rmse11])
        lls.append(ll)
    results_dict[data_directory] = (errors, MC_errors, lls)
        

Loading data and other hyperparameters...
Done.
split 0 of 20.0
Loading file: ./UCI_Datasets/bostonHousing/data/index_train_0.txt
Loading file: ./UCI_Datasets/bostonHousing/data/index_test_0.txt
Number of training examples: 364
Number of validation examples: 91
Number of test examples: 51
Number of train_original examples: 455
2.462648106457164 2.3594571107847875
[0.04627496937384299, 0.24635220848601058, 0.04634022954873033, 0.04634022954873033, 0.19636004096210613, 0.0381463991250333, 0.7639369473279507, 0.7289504036908881, 0.7639369473279507, 0.7639369473279507, 0.645520953479431, 0.7627835447904652, 0.7627835447904652]
split 1 of 20.0
Loading file: ./UCI_Datasets/bostonHousing/data/index_train_1.txt
Loading file: ./UCI_Datasets/bostonHousing/data/index_test_1.txt
Number of training examples: 364
Number of validation examples: 91
Number of test examples: 51
Number of train_original examples: 455
4.638119800565098 2.9133640414355346
[0.7041816419713401, 0.3806168026164208, 0.70418164

In [4]:
results_dict

{'bostonHousing': ([2.462648106457164,
   4.638119800565098,
   2.472703155414779,
   2.6880456260615277,
   4.279479333069791,
   2.5940216123937354,
   1.8463342936976133,
   3.5420335397165434,
   3.9302708785691176,
   4.460086420727231,
   3.4344063414371395,
   2.657031244886552,
   2.761582769121376,
   3.271901417601005,
   2.886777198559651,
   2.934218604054523,
   2.828740328468234,
   3.0383128871079106,
   2.67079433763094,
   4.047576102194134],
  [[0.04627496937384299,
    0.24635220848601058,
    0.04634022954873033,
    0.04634022954873033,
    0.19636004096210613,
    0.0381463991250333,
    0.7639369473279507,
    0.7289504036908881,
    0.7639369473279507,
    0.7639369473279507,
    0.645520953479431,
    0.7627835447904652,
    0.7627835447904652],
   [0.7041816419713401,
    0.3806168026164208,
    0.7041816419713401,
    0.7041816419713401,
    0.35704854453159346,
    0.7009376355494774,
    0.5032679738562091,
    0.6239907727797,
    0.5032679738562091,
    0

In [7]:
"VSGLB 10 models"

'VSGLB 10 models'

In [2]:
import math
import numpy as np
import argparse
import sys
from scipy.stats import norm
def write(X, y, file):
    with open(file, "w") as f:
        for i in range(len(y)):
            f.write("{y}\t{y}\t{x}\n".format(y=y[i], x='\t'.join(map(str, X[i]))))
def writeOOD(X, Y, file):
    y = []
    with open(file, "w") as f:
        for i in range(len(X)):
            y.append(0)
            f.write("{y}\t{y}\t{x}\n".format(y=0, x='\t'.join(map(str, X[i]))))
        for i in range(len(Y)):
            y.append(1)
            f.write("{y}\t{y}\t{x}\n".format(y=1, x='\t'.join(map(str, Y[i]))))
    return np.array(y)
from sklearn.metrics import auc

from sklearn.metrics import roc_auc_score
def Auc(x, y):
    order = np.lexsort((y, x))
    x, y = np.array(x)[order], np.array(y)[order]
    return auc(x, y)


def reject_MSE(targets, preds, measure, measure_name, save_path, pos_label=1, show=True):
    if pos_label != 1:
        measure_loc = -1.0 * measure
    else:
        measure_loc = measure
    preds = np.squeeze(preds)
    # Compute total MSE
    error = (preds - targets)**2
    MSE_0 = np.mean(error)
    # print 'BASE MSE', MSE_0

    # Create array
    array = np.concatenate(
        (preds[:, np.newaxis], targets[:, np.newaxis], error[:, np.newaxis], measure_loc[:, np.newaxis]), axis=1)

    # Results arrays
    results_max = [[0.0, 0.0]]
    results_var = [[0.0, 0.0]]
    results_min = [[0.0, 0.0]]

    optimal_ranking = array[:, 2].argsort()
    sorted_array = array[optimal_ranking]  # Sort by error

    for i in range(1, array.shape[0]):
        x = np.concatenate((sorted_array[:-i, 0], sorted_array[-i:, 1]), axis=0)
        mse = np.mean((x - sorted_array[:, 1])**2)
        # Best rejection
        results_max.append([float(i) / float(array.shape[0]), (MSE_0 - mse) / MSE_0])
        # Random Rejection
        results_min.append([float(i) / float(array.shape[0]), float(i) / float(array.shape[0])])

    uncertainty_ranking = array[:, 3].argsort()
    sorted_array = array[uncertainty_ranking]  # Sort by uncertainty

    for i in range(1, array.shape[0]):
        x = np.concatenate((sorted_array[:-i, 0], sorted_array[-i:, 1]), axis=0)
        mse = np.mean((x - sorted_array[:, 1]) ** 2)
        results_var.append([float(i) / float(array.shape[0]), (MSE_0 - mse) / MSE_0])

    max_auc = Auc([x[0] for x in results_max], [x[1] for x in results_max])
    var_auc = Auc([x[0] for x in results_var], [x[1] for x in results_var])
    min_auc = Auc([x[0] for x in results_min], [x[1] for x in results_min])

#     plt.scatter([x[0] for x in results_max], [x for x in np.asarray(sorted(measure_loc, reverse=True))])
#     plt.xlim(0.0, 1.0)
#     if show == True:
#         plt.savefig(os.path.join(save_path, measure_name), bbox_inches='tight')
#     plt.close()
#     plt.plot([x[0] for x in results_max], [x[1] for x in results_max], '^',
#              [x[0] for x in results_var], [x[1] for x in results_var], 'o',
#              [x[0] for x in results_min], [x[1] for x in results_min], '--')
#     plt.legend(['Optimal-Rejection', 'Model-Rejection', 'Expected Random-Rejection'], loc=4)
#     plt.xlim(0.0, 1.0)
#     plt.ylim(0.0, 1.0)
#     plt.xlabel('Rejection Fraction')
#     plt.ylabel('Pearson Correlation')
#     if show==True:
#         plt.savefig(os.path.join(save_path, "MSE Rejection Curve using " + measure_name), bbox_inches='tight')
#     plt.close()

    AUC_RR = (var_auc - min_auc) / (max_auc - min_auc)
#     if save_path is not None:
#         with open(os.path.join(save_path, 'results.txt'), 'a') as f:
#             f.write('MSE ROC using ' + measure_name + ": " + str(np.round(AUC_RR * 100.0, 1)) + '\n')
    return np.sqrt(MSE_0), AUC_RR, min_auc, var_auc, max_auc
def reject_MSE1(targets, preds, measure, measure_name, save_path, pos_label=1, show=True):
    return 0.0, roc_auc_score(targets, measure), 0.0, 0.0, 0.0
#     return np.sqrt(MSE_0), AUC_RR, min_auc, var_auc, max_auc
approach = "sglb"
params_dict = {}
results_dict = dict()
for data_directory in ["bostonHousing",  "kin8nm", "protein-tertiary-structure", "concrete", "naval-propulsion-plant",  "wine-quality-red", "energy", "power-plant", "yacht"]:
    from subprocess import call
  
    _DATA_DIRECTORY_PATH = "./UCI_Datasets/" + data_directory + "/data/"
    _DROPOUT_RATES_FILE = _DATA_DIRECTORY_PATH + "dropout_rates.txt"
    _TAU_VALUES_FILE = _DATA_DIRECTORY_PATH + "tau_values.txt"
    _DATA_FILE = _DATA_DIRECTORY_PATH + "data.txt"
    _DATA_FILE_OOD = _DATA_DIRECTORY_PATH + "data-mix-ood.txt"
    _HIDDEN_UNITS_FILE = _DATA_DIRECTORY_PATH + "n_hidden.txt"
    _EPOCHS_FILE = _DATA_DIRECTORY_PATH + "n_epochs.txt"
    _INDEX_FEATURES_FILE = _DATA_DIRECTORY_PATH + "index_features.txt"
    _INDEX_TARGET_FILE = _DATA_DIRECTORY_PATH + "index_target.txt"
    _N_SPLITS_FILE = _DATA_DIRECTORY_PATH + "n_splits.txt"
    def _get_index_train_test_path(split_num, train = True):
        """
           Method to generate the path containing the training/test split for the given
           split number (generally from 1 to 20).
           @param split_num      Split number for which the data has to be generated
           @param train          Is true if the data is training data. Else false.
           @return path          Path of the file containing the requried data
        """
        if train:
            return _DATA_DIRECTORY_PATH + "index_train_" + str(split_num) + ".txt"
        else:
            return _DATA_DIRECTORY_PATH + "index_test_" + str(split_num) + ".txt" 


    # We fix the random seed

    np.random.seed(1)

    print ("Loading data and other hyperparameters...")
    # We load the data

    data = np.loadtxt(_DATA_FILE)
    data_ood = np.loadtxt(_DATA_FILE_OOD)


    # We load the indexes for the features and for the target

    index_features = np.loadtxt(_INDEX_FEATURES_FILE)
    index_target = np.loadtxt(_INDEX_TARGET_FILE)

    X = data[ : , [int(i) for i in index_features.tolist()] ]
    Xood = data_ood [1: : , [int(i) for i in index_features.tolist()]]
    y = data[ : , int(index_target.tolist()) ]

    # We iterate over the training test splits

    n_splits = np.loadtxt(_N_SPLITS_FILE)
    print ("Done.")

    errors, MC_errors, lls = [], [], []
    for split in range(int(n_splits)):
        print("split {} of {}".format(split, n_splits))
        # We load the indexes of the training and test sets
        print ('Loading file: ' + _get_index_train_test_path(split, train=True))
        print ('Loading file: ' + _get_index_train_test_path(split, train=False))
        index_train = np.loadtxt(_get_index_train_test_path(split, train=True))
        index_test = np.loadtxt(_get_index_train_test_path(split, train=False))

        X_train = X[ [int(i) for i in index_train.tolist()] ]
        y_train = y[ [int(i) for i in index_train.tolist()] ]

        X_test = X[ [int(i) for i in index_test.tolist()] ]
        y_test = y[ [int(i) for i in index_test.tolist()] ]

        X_train_original = X_train
        y_train_original = y_train
        num_training_examples = int(0.8 * X_train.shape[0])
        X_validation = X_train[num_training_examples:, :]
        y_validation = y_train[num_training_examples:]
        X_train = X_train[0:num_training_examples, :]
        y_train = y_train[0:num_training_examples]
        y_train_mean = y_train.mean()
        y_train_std = y_train.std(ddof=0)
        y_train_normalized = y_train - y_train_mean
        y_validate_normalized = y_validation - y_train_mean
        write(X_train, y_train_normalized, "_train.tsv")
        write(X_validation, y_validate_normalized, "_validation.tsv")
        N = len(y_train)
        beta = N + 0.
        shrinkage = 0.5 / N
        num_ensembles = 1
        num = 10
        # Printing the size of the training, validation and test sets
        print ('Number of training examples: ' + str(X_train.shape[0]))
        print ('Number of validation examples: ' + str(X_validation.shape[0]))
        print ('Number of test examples: ' + str(X_test.shape[0]))
        print ('Number of train_original examples: ' + str(X_train_original.shape[0]))

        # List of hyperparameters which we will try out using grid-search
        depths = [3,4,5]
        lrs = [0.0001, 0.001, 0.01, 0.1]
        rss = [0]
        best_depth = 4
        best_lr = 0.003
        best_rs = 0.1
        iterations = [1000]
        best_ll = float("-inf")
        best_iters = 1000
        for depth in depths:
            for lr in lrs:
                for rs in rss:
                    for iters in iterations:
                        scores = []
                        #iterations_ = []
                        predictions = []
                        for index in range(num_ensembles):
                            !./catboost fit -f _train.tsv -t _validation.tsv  --bootstrap-type Bernoulli --subsample 0.5  --cd pool.cd --use-best-model False --depth $depth --learning-rate $lr --random-strength $rs --leaf-estimation-method Newton --loss-function MultiRMSE:mean=0\;std=$y_train_std --iterations $iters --seed $index -m out_.m --langevin True --diffusion-temperature $beta --model-shrink-rate $shrinkage  | grep best > output__.tsv
                            with open("output__.tsv") as f:
                                lines = f.readlines()
                                scores = [float(line.split('test: ')[-1].split()[0]) for line in lines[-2-num*100:-2]]
                                #iterations_.append(int(lines[-1].split('=')[-1]))
                            
                        arr = np.array(scores)[99::100].astype(np.float128)
                        ll = -np.mean(arr)
                        if ll > best_ll:
                            best_ll = ll
                            best_rs = rs
                            best_lr = lr
                            best_depth = depth
                            best_iters = iters#int(np.mean(iterations_) + 0.5)
        
        X_train = X_train_original
        y_train = y_train_original
        y_train_mean = y_train.mean()
        y_train_std = y_train.std(ddof=0)
        y_train_normalized = y_train - y_train_mean
        y_test_normalized = y_test - y_train_mean
        write(X_train, y_train_normalized, "_train.tsv")
        write(X_test, y_test_normalized, "_test.tsv")
        yood = writeOOD(X_test, Xood, "_ood.tsv")
        N = len(y_train)
        beta = N + 0.
        shrinkage = 0.5 / N
        predictions = []
        for index in range(num_ensembles):
            !./catboost fit -f _train.tsv -t _test.tsv --cd pool.cd  --bootstrap-type Bernoulli --subsample 0.5 --use-best-model False --depth $best_depth --learning-rate $best_lr --random-strength $best_rs --leaf-estimation-method Newton --loss-function MultiRMSE:mean=0\;std=$y_train_std --iterations $best_iters --seed $index -m out_.m --langevin True --diffusion-temperature $beta --model-shrink-rate $shrinkage > output__.tsv
            for i in range(num):
                trees = best_iters - i*100
                !./catboost calc -m out_.m --input-path _ood.tsv -o out_.tsv --cd pool.cd --prediction-type RawFormulaVal --tree-count-limit $trees > stdoutapply.txt
                with open("out_.tsv") as f:
                    tmp = [x.strip().split('\t') for x in f.readlines()[1:]]
                    CC = (1 - best_lr * shrinkage) ** (-i*100)
                    predictions.append([[CC * float(x[1]), np.exp(np.log(y_train_std) + CC * float(x[2]))] for x in tmp])
        ll = 0.0
        for index in range(len(y_test)):
            p = 0.0
            yy = y_test_normalized[index]
            for i in range(num):
                p += norm.pdf((yy - predictions[i][index][0]) / predictions[i][index][1]) / predictions[i][index][1]
            ll += (np.log(num) - np.log(p+1e-6)) / len(y_test)
        preds = np.array([np.mean([x[y][0] for x in predictions]) for y in range(len(y_test))])
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])]) + np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])])
        mse, rmse1, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse2, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        means = np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])
        stds = np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])
        epkl = np.array([np.sum([0.5 * ((s2 ** 2 / s1 ** 2) + (m1-m2)**2 / s1 ** 2 - 1 + 2*np.log(s2/s1)) for m1, s1 in zip(x1, y1) for m2, s2 in zip(x1, y1)]) / len(x1) / max(1, len(x1) - 1) for x1, y1 in zip(means, stds)])
        mse, rmse4, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, epkl, "variance", "anywhere")
        ent = np.array([np.mean(0.5 * np.log(x)) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse3, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, ent, "variance", "anywhere")
        ub = epkl + ent
        mse, rmse5, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, ub, "variance", "anywhere")
        
        y_test = yood
        y_test_normalized = yood
        preds = np.array([np.mean([x[y][0] for x in predictions]) for y in range(len(y_test))])
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])]) + np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse6, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])])
        _, rmse7, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse8, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        means = np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])
        stds = np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])
        epkl = np.array([np.sum([0.5 * ((s2 ** 2 / s1 ** 2) + (m1-m2)**2 / s1 ** 2 - 1 + 2*np.log(s2/s1)) for m1, s1 in zip(x1, y1) for m2, s2 in zip(x1, y1)]) / len(x1) / max(1, len(x1) - 1) for x1, y1 in zip(means, stds)])
        _, rmse10, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, epkl, "variance", "anywhere")
        ent = np.array([np.mean(0.5 * np.log(x)) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse9, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, ent, "variance", "anywhere")
        ub = epkl + ent
        _, rmse11, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, ub, "variance", "anywhere")
        
        print(mse, ll)
        errors.append(mse)
        MC_errors.append([rmse, rmse1, rmse2, rmse3, rmse4, rmse5, rmse6, rmse7, rmse8, rmse9, rmse10, rmse11, rmse11])
        print([rmse, rmse1, rmse2, rmse3, rmse4, rmse5, rmse6, rmse7, rmse8, rmse9, rmse10, rmse11, rmse11])
        lls.append(ll)
    results_dict[data_directory] = (errors, MC_errors, lls)
        

Loading data and other hyperparameters...
Done.
split 0 of 20.0
Loading file: ./UCI_Datasets/bostonHousing/data/index_train_0.txt
Loading file: ./UCI_Datasets/bostonHousing/data/index_test_0.txt
Number of training examples: 364
Number of validation examples: 91
Number of test examples: 51
Number of train_original examples: 455
2.7344117322971386 2.3593616568991282
[0.32326885294205515, 0.15178351695815775, 0.30512900029576395, 0.29514329155223534, 0.2457052411213505, 0.2776754500591142, 0.5832372164552095, 0.40561322568242986, 0.5943867743175701, 0.6501345636293733, 0.27297193387158786, 0.4183006535947712, 0.4183006535947712]
split 1 of 20.0
Loading file: ./UCI_Datasets/bostonHousing/data/index_train_1.txt
Loading file: ./UCI_Datasets/bostonHousing/data/index_test_1.txt
Number of training examples: 364
Number of validation examples: 91
Number of test examples: 51
Number of train_original examples: 455
2.9830481020384516 2.444323963683725
[0.23582432003026368, 0.19972292168189032, 0.220

In [3]:
results_dict

{'bostonHousing': ([2.7344117322971386,
   2.9830481020384516,
   2.740315645751129,
   3.2020043391614847,
   4.788309352202701,
   2.928665362003816,
   1.7747095667462336,
   4.23490866489459,
   3.927564534018205,
   4.745688032969436,
   3.587075317893696,
   3.284941567354195,
   3.11384178097397,
   3.638802514750611,
   3.115138771305764,
   3.403732368022331,
   2.9889163885634473,
   3.2860168496038114,
   2.9695023687443185,
   4.667569095214155],
  [[0.32326885294205515,
    0.15178351695815775,
    0.30512900029576395,
    0.29514329155223534,
    0.2457052411213505,
    0.2776754500591142,
    0.5832372164552095,
    0.40561322568242986,
    0.5943867743175701,
    0.6501345636293733,
    0.27297193387158786,
    0.4183006535947712,
    0.4183006535947712],
   [0.23582432003026368,
    0.19972292168189032,
    0.22095333316146568,
    0.23426247942101794,
    -0.021531094886956845,
    0.144356515318223,
    0.5278738946559016,
    0.40868896578239133,
    0.5524798154555

In [8]:
"SGLB 10 models"

'SGLB 10 models'

In [4]:
import math
import numpy as np
import argparse
import sys
from scipy.stats import norm
def write(X, y, file):
    with open(file, "w") as f:
        for i in range(len(y)):
            f.write("{y}\t{y}\t{x}\n".format(y=y[i], x='\t'.join(map(str, X[i]))))
def writeOOD(X, Y, file):
    y = []
    with open(file, "w") as f:
        for i in range(len(X)):
            y.append(0)
            f.write("{y}\t{y}\t{x}\n".format(y=0, x='\t'.join(map(str, X[i]))))
        for i in range(len(Y)):
            y.append(1)
            f.write("{y}\t{y}\t{x}\n".format(y=1, x='\t'.join(map(str, Y[i]))))
    return np.array(y)
from sklearn.metrics import auc

from sklearn.metrics import roc_auc_score
def Auc(x, y):
    order = np.lexsort((y, x))
    x, y = np.array(x)[order], np.array(y)[order]
    return auc(x, y)


def reject_MSE(targets, preds, measure, measure_name, save_path, pos_label=1, show=True):
    if pos_label != 1:
        measure_loc = -1.0 * measure
    else:
        measure_loc = measure
    preds = np.squeeze(preds)
    # Compute total MSE
    error = (preds - targets)**2
    MSE_0 = np.mean(error)
    # print 'BASE MSE', MSE_0

    # Create array
    array = np.concatenate(
        (preds[:, np.newaxis], targets[:, np.newaxis], error[:, np.newaxis], measure_loc[:, np.newaxis]), axis=1)

    # Results arrays
    results_max = [[0.0, 0.0]]
    results_var = [[0.0, 0.0]]
    results_min = [[0.0, 0.0]]

    optimal_ranking = array[:, 2].argsort()
    sorted_array = array[optimal_ranking]  # Sort by error

    for i in range(1, array.shape[0]):
        x = np.concatenate((sorted_array[:-i, 0], sorted_array[-i:, 1]), axis=0)
        mse = np.mean((x - sorted_array[:, 1])**2)
        # Best rejection
        results_max.append([float(i) / float(array.shape[0]), (MSE_0 - mse) / MSE_0])
        # Random Rejection
        results_min.append([float(i) / float(array.shape[0]), float(i) / float(array.shape[0])])

    uncertainty_ranking = array[:, 3].argsort()
    sorted_array = array[uncertainty_ranking]  # Sort by uncertainty

    for i in range(1, array.shape[0]):
        x = np.concatenate((sorted_array[:-i, 0], sorted_array[-i:, 1]), axis=0)
        mse = np.mean((x - sorted_array[:, 1]) ** 2)
        results_var.append([float(i) / float(array.shape[0]), (MSE_0 - mse) / MSE_0])

    max_auc = Auc([x[0] for x in results_max], [x[1] for x in results_max])
    var_auc = Auc([x[0] for x in results_var], [x[1] for x in results_var])
    min_auc = Auc([x[0] for x in results_min], [x[1] for x in results_min])

    AUC_RR = (var_auc - min_auc) / (max_auc - min_auc)
    return np.sqrt(MSE_0), AUC_RR, min_auc, var_auc, max_auc
def reject_MSE1(targets, preds, measure, measure_name, save_path, pos_label=1, show=True):
    return 0.0, roc_auc_score(targets, measure), 0.0, 0.0, 0.0
#     return np.sqrt(MSE_0), AUC_RR, min_auc, var_auc, max_auc
approach = "sglb"
params_dict = {}
results_dict = dict()
for data_directory in ["bostonHousing",  "kin8nm", "protein-tertiary-structure", "concrete", "naval-propulsion-plant",  "wine-quality-red", "energy", "power-plant", "yacht"]:
    from subprocess import call
  
    _DATA_DIRECTORY_PATH = "./UCI_Datasets/" + data_directory + "/data/"
    _DROPOUT_RATES_FILE = _DATA_DIRECTORY_PATH + "dropout_rates.txt"
    _TAU_VALUES_FILE = _DATA_DIRECTORY_PATH + "tau_values.txt"
    _DATA_FILE = _DATA_DIRECTORY_PATH + "data.txt"
    _DATA_FILE_OOD = _DATA_DIRECTORY_PATH + "data-mix-ood.txt"
    _HIDDEN_UNITS_FILE = _DATA_DIRECTORY_PATH + "n_hidden.txt"
    _EPOCHS_FILE = _DATA_DIRECTORY_PATH + "n_epochs.txt"
    _INDEX_FEATURES_FILE = _DATA_DIRECTORY_PATH + "index_features.txt"
    _INDEX_TARGET_FILE = _DATA_DIRECTORY_PATH + "index_target.txt"
    _N_SPLITS_FILE = _DATA_DIRECTORY_PATH + "n_splits.txt"
    def _get_index_train_test_path(split_num, train = True):
        """
           Method to generate the path containing the training/test split for the given
           split number (generally from 1 to 20).
           @param split_num      Split number for which the data has to be generated
           @param train          Is true if the data is training data. Else false.
           @return path          Path of the file containing the requried data
        """
        if train:
            return _DATA_DIRECTORY_PATH + "index_train_" + str(split_num) + ".txt"
        else:
            return _DATA_DIRECTORY_PATH + "index_test_" + str(split_num) + ".txt" 


    # We fix the random seed

    np.random.seed(1)

    print ("Loading data and other hyperparameters...")
    # We load the data

    data = np.loadtxt(_DATA_FILE)
    data_ood = np.loadtxt(_DATA_FILE_OOD)


    # We load the indexes for the features and for the target

    index_features = np.loadtxt(_INDEX_FEATURES_FILE)
    index_target = np.loadtxt(_INDEX_TARGET_FILE)

    X = data[ : , [int(i) for i in index_features.tolist()] ]
    Xood = data_ood [1: : , [int(i) for i in index_features.tolist()]]
    y = data[ : , int(index_target.tolist()) ]

    # We iterate over the training test splits

    n_splits = np.loadtxt(_N_SPLITS_FILE)
    print ("Done.")

    errors, MC_errors, lls = [], [], []
    for split in range(int(n_splits)):
        print("split {} of {}".format(split, n_splits))
        # We load the indexes of the training and test sets
        print ('Loading file: ' + _get_index_train_test_path(split, train=True))
        print ('Loading file: ' + _get_index_train_test_path(split, train=False))
        index_train = np.loadtxt(_get_index_train_test_path(split, train=True))
        index_test = np.loadtxt(_get_index_train_test_path(split, train=False))

        X_train = X[ [int(i) for i in index_train.tolist()] ]
        y_train = y[ [int(i) for i in index_train.tolist()] ]

        X_test = X[ [int(i) for i in index_test.tolist()] ]
        y_test = y[ [int(i) for i in index_test.tolist()] ]

        X_train_original = X_train
        y_train_original = y_train
        num_training_examples = int(0.8 * X_train.shape[0])
        X_validation = X_train[num_training_examples:, :]
        y_validation = y_train[num_training_examples:]
        X_train = X_train[0:num_training_examples, :]
        y_train = y_train[0:num_training_examples]
        y_train_mean = y_train.mean()
        y_train_std = y_train.std(ddof=0)
        y_train_normalized = y_train - y_train_mean
        y_validate_normalized = y_validation - y_train_mean
        write(X_train, y_train_normalized, "_train.tsv")
        write(X_validation, y_validate_normalized, "_validation.tsv")
        N = len(y_train)
        beta = N + 0.
        shrinkage = 0.5 / N
        num_ensembles = 10
        num = 1
        # Printing the size of the training, validation and test sets
        print ('Number of training examples: ' + str(X_train.shape[0]))
        print ('Number of validation examples: ' + str(X_validation.shape[0]))
        print ('Number of test examples: ' + str(X_test.shape[0]))
        print ('Number of train_original examples: ' + str(X_train_original.shape[0]))

        # List of hyperparameters which we will try out using grid-search
        
        best_depth = params_dict[data_directory][split]["depth"]
        best_lr = params_dict[data_directory][split]["lr"]
        best_rs = params_dict[data_directory][split]["rs"]
        best_ll = float("-inf")
        best_iters = params_dict[data_directory][split]["iters"]
        
        X_train = X_train_original
        y_train = y_train_original
        y_train_mean = y_train.mean()
        y_train_std = y_train.std(ddof=0)
        y_train_normalized = y_train - y_train_mean
        y_test_normalized = y_test - y_train_mean
        write(X_train, y_train_normalized, "_train.tsv")
        write(X_test, y_test_normalized, "_test.tsv")
        yood = writeOOD(X_test, Xood, "_ood.tsv")
        N = len(y_train)
        beta = N + 0.
        shrinkage = 0.5 / N
        predictions = []
        for index in range(num_ensembles):
            !./catboost fit -f _train.tsv -t _test.tsv --cd pool.cd  --bootstrap-type Bernoulli --subsample 0.5 --use-best-model False --depth $best_depth --learning-rate $best_lr --random-strength $best_rs --leaf-estimation-method Newton --loss-function MultiRMSE:mean=0\;std=$y_train_std --iterations $best_iters --seed $index -m out_.m --langevin True --diffusion-temperature $beta --model-shrink-rate $shrinkage  > output__.tsv
            for i in range(num):
                trees = best_iters - i * 100
                !./catboost calc -m out_.m --input-path _ood.tsv -o out_.tsv --cd pool.cd --prediction-type RawFormulaVal --tree-count-limit $trees > stdoutapply.txt
                with open("out_.tsv") as f:
                    tmp = [x.strip().split('\t') for x in f.readlines()[1:]]
                    CC = (1 - best_lr * shrinkage) ** (-i * 100)
                    predictions.append([[CC * float(x[1]), np.exp(np.log(y_train_std) + CC * float(x[2]))] for x in tmp])
        ll = 0.0
        for index in range(len(y_test)):
            p = 0.0
            yy = y_test_normalized[index]
            for i in range(max(num_ensembles, num)):
                p += norm.pdf((yy - predictions[i][index][0]) / predictions[i][index][1]) / predictions[i][index][1] / max(num_ensembles, num)
            ll += (np.log(num) - np.log(p+1e-6)) / len(y_test)
        preds = np.array([np.mean([x[y][0] for x in predictions]) for y in range(len(y_test))])
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])]) + np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])])
        mse, rmse1, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse2, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        means = np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])
        stds = np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])
        epkl = np.array([np.sum([0.5 * ((s2 ** 2 / s1 ** 2) + (m1-m2)**2 / s1 ** 2 - 1 + 2*np.log(s2/s1)) for m1, s1 in zip(x1, y1) for m2, s2 in zip(x1, y1)]) / len(x1) / max(1, len(x1) - 1) for x1, y1 in zip(means, stds)])
        mse, rmse4, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, epkl, "variance", "anywhere")
        ent = np.array([np.mean(0.5 * np.log(x)) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse3, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, ent, "variance", "anywhere")
        ub = epkl + ent
        mse, rmse5, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, ub, "variance", "anywhere")
        
        y_test = yood
        y_test_normalized = yood
        preds = np.array([np.mean([x[y][0] for x in predictions]) for y in range(len(y_test))])
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])]) + np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse6, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])])
        _, rmse7, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse8, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        means = np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])
        stds = np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])
        epkl = np.array([np.sum([0.5 * ((s2 ** 2 / s1 ** 2) + (m1-m2)**2 / s1 ** 2 - 1 + 2*np.log(s2/s1)) for m1, s1 in zip(x1, y1) for m2, s2 in zip(x1, y1)]) / len(x1) / max(1, len(x1) - 1) for x1, y1 in zip(means, stds)])
        _, rmse10, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, epkl, "variance", "anywhere")
        ent = np.array([np.mean(0.5 * np.log(x)) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse9, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, ent, "variance", "anywhere")
        ub = epkl + ent
        _, rmse11, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, ub, "variance", "anywhere")
        
        print(mse, ll)
        errors.append(mse)
        MC_errors.append([rmse, rmse1, rmse2, rmse3, rmse4, rmse5, rmse6, rmse7, rmse8, rmse9, rmse10, rmse11, rmse11])
        print([rmse, rmse1, rmse2, rmse3, rmse4, rmse5, rmse6, rmse7, rmse8, rmse9, rmse10, rmse11, rmse11])
        lls.append(ll)
    results_dict[data_directory] = (errors, MC_errors, lls)
        

Loading data and other hyperparameters...
Done.
split 0 of 20.0
Loading file: ./UCI_Datasets/bostonHousing/data/index_train_0.txt
Loading file: ./UCI_Datasets/bostonHousing/data/index_test_0.txt
Number of training examples: 364
Number of validation examples: 91
Number of test examples: 51
Number of train_original examples: 455
2.4586571651309495 2.3585621984611356
[0.07927297011251455, 0.21387215511927873, 0.07858848457885924, 0.07862159298067936, 0.12611180515210985, 0.059992344138919976, 0.7585544021530181, 0.8358323721645521, 0.7585544021530181, 0.7585544021530181, 0.7954632833525567, 0.7685505574778931, 0.7685505574778931]
split 1 of 20.0
Loading file: ./UCI_Datasets/bostonHousing/data/index_train_1.txt
Loading file: ./UCI_Datasets/bostonHousing/data/index_test_1.txt
Number of training examples: 364
Number of validation examples: 91
Number of test examples: 51
Number of train_original examples: 455
4.638377354298673 2.913757193439332
[0.699564669206924, 0.41170990383271977, 0.69956

In [5]:
results_dict

{'bostonHousing': ([2.4586571651309495,
   4.638377354298673,
   2.4617806110454796,
   2.691004958505744,
   4.250743352302808,
   2.6056744694881453,
   1.8348837222008685,
   3.5593362296018274,
   3.923297206193253,
   4.448595824723008,
   3.4414992150397006,
   2.929220328696396,
   2.767668467374837,
   3.2530418730914668,
   2.8956270936205026,
   2.9253862955258136,
   2.8258416502391115,
   3.0605062056811083,
   2.67944212110916,
   4.020290838394368],
  [[0.07927297011251455,
    0.21387215511927873,
    0.07858848457885924,
    0.07862159298067936,
    0.12611180515210985,
    0.059992344138919976,
    0.7585544021530181,
    0.8358323721645521,
    0.7585544021530181,
    0.7585544021530181,
    0.7954632833525567,
    0.7685505574778931,
    0.7685505574778931],
   [0.699564669206924,
    0.41170990383271977,
    0.699564669206924,
    0.699564669206924,
    0.3415294096746718,
    0.699564669206924,
    0.502883506343714,
    0.6405228758169934,
    0.502883506343714,
 

In [9]:
"SGB 1 model"

'SGB 1 model'

In [6]:
import math
import numpy as np
import argparse
import sys
from scipy.stats import norm
def write(X, y, file):
    with open(file, "w") as f:
        for i in range(len(y)):
            f.write("{y}\t{y}\t{x}\n".format(y=y[i], x='\t'.join(map(str, X[i]))))
def writeOOD(X, Y, file):
    y = []
    with open(file, "w") as f:
        for i in range(len(X)):
            y.append(0)
            f.write("{y}\t{y}\t{x}\n".format(y=0, x='\t'.join(map(str, X[i]))))
        for i in range(len(Y)):
            y.append(1)
            f.write("{y}\t{y}\t{x}\n".format(y=1, x='\t'.join(map(str, Y[i]))))
    return np.array(y)
from sklearn.metrics import auc

from sklearn.metrics import roc_auc_score
def Auc(x, y):
    order = np.lexsort((y, x))
    x, y = np.array(x)[order], np.array(y)[order]
    return auc(x, y)


def reject_MSE(targets, preds, measure, measure_name, save_path, pos_label=1, show=True):
    if pos_label != 1:
        measure_loc = -1.0 * measure
    else:
        measure_loc = measure
    preds = np.squeeze(preds)
    # Compute total MSE
    error = (preds - targets)**2
    MSE_0 = np.mean(error)
    # print 'BASE MSE', MSE_0

    # Create array
    array = np.concatenate(
        (preds[:, np.newaxis], targets[:, np.newaxis], error[:, np.newaxis], measure_loc[:, np.newaxis]), axis=1)

    # Results arrays
    results_max = [[0.0, 0.0]]
    results_var = [[0.0, 0.0]]
    results_min = [[0.0, 0.0]]

    optimal_ranking = array[:, 2].argsort()
    sorted_array = array[optimal_ranking]  # Sort by error

    for i in range(1, array.shape[0]):
        x = np.concatenate((sorted_array[:-i, 0], sorted_array[-i:, 1]), axis=0)
        mse = np.mean((x - sorted_array[:, 1])**2)
        # Best rejection
        results_max.append([float(i) / float(array.shape[0]), (MSE_0 - mse) / MSE_0])
        # Random Rejection
        results_min.append([float(i) / float(array.shape[0]), float(i) / float(array.shape[0])])

    uncertainty_ranking = array[:, 3].argsort()
    sorted_array = array[uncertainty_ranking]  # Sort by uncertainty

    for i in range(1, array.shape[0]):
        x = np.concatenate((sorted_array[:-i, 0], sorted_array[-i:, 1]), axis=0)
        mse = np.mean((x - sorted_array[:, 1]) ** 2)
        results_var.append([float(i) / float(array.shape[0]), (MSE_0 - mse) / MSE_0])

    max_auc = Auc([x[0] for x in results_max], [x[1] for x in results_max])
    var_auc = Auc([x[0] for x in results_var], [x[1] for x in results_var])
    min_auc = Auc([x[0] for x in results_min], [x[1] for x in results_min])

#     plt.scatter([x[0] for x in results_max], [x for x in np.asarray(sorted(measure_loc, reverse=True))])
#     plt.xlim(0.0, 1.0)
#     if show == True:
#         plt.savefig(os.path.join(save_path, measure_name), bbox_inches='tight')
#     plt.close()
#     plt.plot([x[0] for x in results_max], [x[1] for x in results_max], '^',
#              [x[0] for x in results_var], [x[1] for x in results_var], 'o',
#              [x[0] for x in results_min], [x[1] for x in results_min], '--')
#     plt.legend(['Optimal-Rejection', 'Model-Rejection', 'Expected Random-Rejection'], loc=4)
#     plt.xlim(0.0, 1.0)
#     plt.ylim(0.0, 1.0)
#     plt.xlabel('Rejection Fraction')
#     plt.ylabel('Pearson Correlation')
#     if show==True:
#         plt.savefig(os.path.join(save_path, "MSE Rejection Curve using " + measure_name), bbox_inches='tight')
#     plt.close()

    AUC_RR = (var_auc - min_auc) / (max_auc - min_auc)
#     if save_path is not None:
#         with open(os.path.join(save_path, 'results.txt'), 'a') as f:
#             f.write('MSE ROC using ' + measure_name + ": " + str(np.round(AUC_RR * 100.0, 1)) + '\n')
    return np.sqrt(MSE_0), AUC_RR, min_auc, var_auc, max_auc
def reject_MSE1(targets, preds, measure, measure_name, save_path, pos_label=1, show=True):
    return 0.0, roc_auc_score(targets, measure), 0.0, 0.0, 0.0
#     return np.sqrt(MSE_0), AUC_RR, min_auc, var_auc, max_auc
approach = "sglb"
params_dict = {}
results_dict = dict()
for data_directory in ["bostonHousing",  "kin8nm", "protein-tertiary-structure", "concrete", "naval-propulsion-plant",  "wine-quality-red", "energy", "power-plant", "yacht"]:
    from subprocess import call
  
    _DATA_DIRECTORY_PATH = "./UCI_Datasets/" + data_directory + "/data/"
    _DROPOUT_RATES_FILE = _DATA_DIRECTORY_PATH + "dropout_rates.txt"
    _TAU_VALUES_FILE = _DATA_DIRECTORY_PATH + "tau_values.txt"
    _DATA_FILE = _DATA_DIRECTORY_PATH + "data.txt"
    _DATA_FILE_OOD = _DATA_DIRECTORY_PATH + "data-mix-ood.txt"
    _HIDDEN_UNITS_FILE = _DATA_DIRECTORY_PATH + "n_hidden.txt"
    _EPOCHS_FILE = _DATA_DIRECTORY_PATH + "n_epochs.txt"
    _INDEX_FEATURES_FILE = _DATA_DIRECTORY_PATH + "index_features.txt"
    _INDEX_TARGET_FILE = _DATA_DIRECTORY_PATH + "index_target.txt"
    _N_SPLITS_FILE = _DATA_DIRECTORY_PATH + "n_splits.txt"
    def _get_index_train_test_path(split_num, train = True):
        """
           Method to generate the path containing the training/test split for the given
           split number (generally from 1 to 20).
           @param split_num      Split number for which the data has to be generated
           @param train          Is true if the data is training data. Else false.
           @return path          Path of the file containing the requried data
        """
        if train:
            return _DATA_DIRECTORY_PATH + "index_train_" + str(split_num) + ".txt"
        else:
            return _DATA_DIRECTORY_PATH + "index_test_" + str(split_num) + ".txt" 


    # We fix the random seed

    np.random.seed(1)

    print ("Loading data and other hyperparameters...")
    # We load the data

    data = np.loadtxt(_DATA_FILE)
    data_ood = np.loadtxt(_DATA_FILE_OOD)


    # We load the indexes for the features and for the target

    index_features = np.loadtxt(_INDEX_FEATURES_FILE)
    index_target = np.loadtxt(_INDEX_TARGET_FILE)

    X = data[ : , [int(i) for i in index_features.tolist()] ]
    Xood = data_ood [1: : , [int(i) for i in index_features.tolist()]]
    y = data[ : , int(index_target.tolist()) ]

    # We iterate over the training test splits

    n_splits = np.loadtxt(_N_SPLITS_FILE)
    print ("Done.")

    errors, MC_errors, lls = [], [], []
    for split in range(int(n_splits)):
        print("split {} of {}".format(split, n_splits))
        # We load the indexes of the training and test sets
        print ('Loading file: ' + _get_index_train_test_path(split, train=True))
        print ('Loading file: ' + _get_index_train_test_path(split, train=False))
        index_train = np.loadtxt(_get_index_train_test_path(split, train=True))
        index_test = np.loadtxt(_get_index_train_test_path(split, train=False))

        X_train = X[ [int(i) for i in index_train.tolist()] ]
        y_train = y[ [int(i) for i in index_train.tolist()] ]

        X_test = X[ [int(i) for i in index_test.tolist()] ]
        y_test = y[ [int(i) for i in index_test.tolist()] ]

        X_train_original = X_train
        y_train_original = y_train
        num_training_examples = int(0.8 * X_train.shape[0])
        X_validation = X_train[num_training_examples:, :]
        y_validation = y_train[num_training_examples:]
        X_train = X_train[0:num_training_examples, :]
        y_train = y_train[0:num_training_examples]
        y_train_mean = y_train.mean()
        y_train_std = y_train.std(ddof=0)
        y_train_normalized = y_train - y_train_mean
        y_validate_normalized = y_validation - y_train_mean
        write(X_train, y_train_normalized, "_train.tsv")
        write(X_validation, y_validate_normalized, "_validation.tsv")
        N = len(y_train)
        beta = N + 0.
        shrinkage = 0.5 / N
        num_ensembles = 1
        num = 1
        # Printing the size of the training, validation and test sets
        print ('Number of training examples: ' + str(X_train.shape[0]))
        print ('Number of validation examples: ' + str(X_validation.shape[0]))
        print ('Number of test examples: ' + str(X_test.shape[0]))
        print ('Number of train_original examples: ' + str(X_train_original.shape[0]))

        # List of hyperparameters which we will try out using grid-search
        
        best_depth = params_dict[data_directory][split]["depth"]
        best_lr = params_dict[data_directory][split]["lr"]
        best_rs = params_dict[data_directory][split]["rs"]
        best_ll = float("-inf")
        best_iters = params_dict[data_directory][split]["iters"]
        
        X_train = X_train_original
        y_train = y_train_original
        y_train_mean = y_train.mean()
        y_train_std = y_train.std(ddof=0)
        y_train_normalized = y_train - y_train_mean
        y_test_normalized = y_test - y_train_mean
        write(X_train, y_train_normalized, "_train.tsv")
        write(X_test, y_test_normalized, "_test.tsv")
        yood = writeOOD(X_test, Xood, "_ood.tsv")
        N = len(y_train)
        beta = N + 0.
        shrinkage = 0.5 / N
        predictions = []
        for index in range(num_ensembles):
            !./catboost fit -f _train.tsv -t _test.tsv --cd pool.cd  --bootstrap-type Bernoulli --subsample 0.5 --use-best-model False --depth $best_depth --learning-rate $best_lr --random-strength $best_rs --leaf-estimation-method Newton --loss-function MultiRMSE:mean=0\;std=$y_train_std --iterations $best_iters --seed $index -m out_.m --langevin True --diffusion-temperature $beta --model-shrink-rate $shrinkage  > output__.tsv
            for i in range(num):
                trees = best_iters - i * 100
                !./catboost calc -m out_.m --input-path _ood.tsv -o out_.tsv --cd pool.cd --prediction-type RawFormulaVal --tree-count-limit $trees > stdoutapply.txt
                with open("out_.tsv") as f:
                    tmp = [x.strip().split('\t') for x in f.readlines()[1:]]
                    CC = (1 - best_lr * shrinkage) ** (-i * 100)
                    predictions.append([[CC * float(x[1]), np.exp(np.log(y_train_std) + CC * float(x[2]))] for x in tmp])
        ll = 0.0
        for index in range(len(y_test)):
            p = 0.0
            yy = y_test_normalized[index]
            for i in range(max(num_ensembles, num)):
                p += norm.pdf((yy - predictions[i][index][0]) / predictions[i][index][1]) / predictions[i][index][1] / max(num_ensembles, num)
            ll += (np.log(num) - np.log(p+1e-6)) / len(y_test)
        preds = np.array([np.mean([x[y][0] for x in predictions]) for y in range(len(y_test))])
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])]) + np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])])
        mse, rmse1, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse2, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        means = np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])
        stds = np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])
        epkl = np.array([np.sum([0.5 * ((s2 ** 2 / s1 ** 2) + (m1-m2)**2 / s1 ** 2 - 1 + 2*np.log(s2/s1)) for m1, s1 in zip(x1, y1) for m2, s2 in zip(x1, y1)]) / len(x1) / max(1, len(x1) - 1) for x1, y1 in zip(means, stds)])
        mse, rmse4, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, epkl, "variance", "anywhere")
        ent = np.array([np.mean(0.5 * np.log(x)) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        mse, rmse3, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, ent, "variance", "anywhere")
        ub = epkl + ent
        mse, rmse5, min_auc, var_auc, max_auc = reject_MSE(y_test_normalized, preds, ub, "variance", "anywhere")
        
        y_test = yood
        y_test_normalized = yood
        preds = np.array([np.mean([x[y][0] for x in predictions]) for y in range(len(y_test))])
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])]) + np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse6, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.var(x, ddof=0) for x in np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])])
        _, rmse7, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        uncertainity = np.array([np.mean(x**2) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse8, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, uncertainity, "variance", "anywhere")
        means = np.array([[x[y][0] for x in predictions] for y in range(len(y_test))])
        stds = np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])
        epkl = np.array([np.sum([0.5 * ((s2 ** 2 / s1 ** 2) + (m1-m2)**2 / s1 ** 2 - 1 + 2*np.log(s2/s1)) for m1, s1 in zip(x1, y1) for m2, s2 in zip(x1, y1)]) / len(x1) / max(1, len(x1) - 1) for x1, y1 in zip(means, stds)])
        _, rmse10, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, epkl, "variance", "anywhere")
        ent = np.array([np.mean(0.5 * np.log(x)) for x in np.array([[x[y][1] for x in predictions] for y in range(len(y_test))])])
        _, rmse9, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, ent, "variance", "anywhere")
        ub = epkl + ent
        _, rmse11, min_auc, var_auc, max_auc = reject_MSE1(y_test_normalized, preds, ub, "variance", "anywhere")
        
        print(mse, ll)
        errors.append(mse)
        MC_errors.append([rmse, rmse1, rmse2, rmse3, rmse4, rmse5, rmse6, rmse7, rmse8, rmse9, rmse10, rmse11, rmse11])
        print([rmse, rmse1, rmse2, rmse3, rmse4, rmse5, rmse6, rmse7, rmse8, rmse9, rmse10, rmse11, rmse11])
        lls.append(ll)
    results_dict[data_directory] = (errors, MC_errors, lls)
        

Loading data and other hyperparameters...
Done.
split 0 of 20.0
Loading file: ./UCI_Datasets/bostonHousing/data/index_train_0.txt
Loading file: ./UCI_Datasets/bostonHousing/data/index_test_0.txt
Number of training examples: 364
Number of validation examples: 91
Number of test examples: 51
Number of train_original examples: 455
2.4879583174784123 2.364719629147714
[0.10932604408236604, 0.1846719294489838, 0.10932604408236604, 0.10932604408236604, 0.1846719294489838, 0.10932604408236604, 0.7608612072279892, 0.5, 0.7608612072279892, 0.7608612072279892, 0.5, 0.7608612072279892, 0.7608612072279892]
split 1 of 20.0
Loading file: ./UCI_Datasets/bostonHousing/data/index_train_1.txt
Loading file: ./UCI_Datasets/bostonHousing/data/index_test_1.txt
Number of training examples: 364
Number of validation examples: 91
Number of test examples: 51
Number of train_original examples: 455
4.651458855923829 2.9158787589177204
[0.6937242684032631, -0.12807153544540797, 0.6937242684032631, 0.6937242684032631

In [7]:
results_dict

{'bostonHousing': ([2.4879583174784123,
   4.651458855923829,
   2.399076243499079,
   2.6765624000675188,
   4.299457560404111,
   2.615683939517051,
   1.9091780845812814,
   3.5824904387043137,
   3.973220346230289,
   4.4907424150356325,
   3.440331442575011,
   2.945171146648961,
   2.7306867844003366,
   3.2760435111125137,
   2.9071080193749017,
   2.927642764108911,
   2.8057790336163073,
   3.05833471137166,
   2.651982200083864,
   4.008019358085155],
  [[0.10932604408236604,
    0.1846719294489838,
    0.10932604408236604,
    0.10932604408236604,
    0.1846719294489838,
    0.10932604408236604,
    0.7608612072279892,
    0.5,
    0.7608612072279892,
    0.7608612072279892,
    0.5,
    0.7608612072279892,
    0.7608612072279892],
   [0.6937242684032631,
    -0.12807153544540797,
    0.6937242684032631,
    0.6937242684032631,
    -0.12807153544540797,
    0.6937242684032631,
    0.5040369088811996,
    0.5,
    0.5040369088811996,
    0.5040369088811996,
    0.5,
    0.504